# __STRATEGIE D'ANALYSE BIOINFORMATIQUE - MENTORING PROJECT__

__AGMIMONHAN Attolou Raoul, NAME Pakyendou Estel__

__Tuteurs: Aurore COMTE & Sebastien RAVEL__

Jupyter inspired by the model created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)
***

# <span style="color: #006E7F">Table of contents</span>
<a class="anchor" id="home"></a>


[PRACTICE IV - SNP calling](#SNP)
   * [Indexation reference genome `samtools faidx`](#refindex)
   * [Generer bcf file `bcftools mpileup`](#bcftools)
   * [Calling `bcftools call`](#calling)
   * [SNP statistiques `bcftools stats`](#stats)
   * [SNP annotation](#annotation)
   
   

***

# __Practice IV - SNP calling__

## __IV.1. Indexation reference genome__

In [ ]:
## Se deplacer dans le répertoire de la référennce

cd /scratch/MOryzae/REF

In [ ]:
# Charger le module samtools 

module load samtools/1.18


In [ ]:
## Indexation de la référence utilisant samtools faidx

samtools faidx /scratch/MOryzae/REF/MOryzae_genomic.fna

In [ ]:
## Vérifier le contenu du répertoire et les fichiers générés

ls -lh

## __IV.2. RUN__

In [ ]:
## Se déplacer dans le répertoire créé

cd /scratch/MOryzae/SCRIPTS

In [ ]:
## Ouvrir l'éditeur de texte nano

nano snp_calling_pipeline.sh

In [ ]:
#!/bin/bash

#SBATCH -c 16
#SBATCH --nodelist=node20

#################################################

########### Execution Command ##################

# Define file paths
SORTED_PATH="/scratch/MOryzae/MAPPING/bam_mapped_sort"
BCF_PATH="/scratch/MOryzae/SNP/bcf_files/all_samples.bcf"
VCF_PATH="/scratch/MOryzae/SNP/vcf_files/all_samples.vcf"
REF_GENOME="/scratch/MOryzae/REF/MOryzae_genomic.fna"
SNP_STATS_DIR="/scratch/MOryzae/SNP/stats"
SNP_FILE="/scratch/MOryzae/SNP/vcf_files/all_samples_snp.vcf"
OUTPUT_VCF="${VCF_PATH}.gz"
OUTPUT_VCF2="${SNP_FILE}.gz"
ALLELE_FREQ_PATH="/scratch/MOryzae/SNP/allele_frequence"

# Load necessary modules
module load samtools/1.18
module load bcftools/1.18
module load vcftools/0.1.16
module load htslib/1.19

# Create directories if necessary
mkdir -p /scratch/MOryzae/SNP/bcf_files /scratch/MOryzae/SNP/vcf_files "$SNP_STATS_DIR" "$ALLELE_FREQ_PATH"

# Step 1: Generate BCF file
echo -e "######################\nGenerating BCF file"
bcftools mpileup --threads 16 -f "$REF_GENOME" -O b -o "$BCF_PATH" "$SORTED_PATH"/*.mappedpaired.sorted.bam || {
    echo "Error: Failed to generate BCF file" >&2
    exit 1
}

# Step 2: Variant calling
echo -e "######################\nVariant calling"
bcftools call --threads 16 -v -c -o "$VCF_PATH" "$BCF_PATH" || {
    echo "Error: Variant calling failed" >&2
    exit 1
}

# Step 3: Generate SNP statistics
echo -e "######################\nGenerating SNP statistics"
bcftools stats "$VCF_PATH" > "$SNP_STATS_DIR/all_samples_SNP_statistics.txt" || {
    echo "Error: Failed to generate SNP statistics" >&2
    exit 1
}

# Step 4: Filter to keep only SNPs
echo -e "######################\nFiltering SNPs"
bcftools view -v snps "$VCF_PATH" -o "$SNP_FILE" || {
    echo "Error: Filtering SNPs failed" >&2
    exit 1
}

# Step 5: Compress and index the VCF file
echo -e "######################\nCompressing and indexing VCF files"
bgzip -c "$VCF_PATH" > "$OUTPUT_VCF" || {
    echo "Error: Compression of VCF file failed" >&2
    exit 1
}
bgzip -c "$SNP_FILE" > "$OUTPUT_VCF2" || {
    echo "Error: Compression of SNP file failed" >&2
    exit 1
}
bcftools index "$OUTPUT_VCF" || {
    echo "Error: Indexing VCF file failed" >&2
    exit 1
}
bcftools index "$OUTPUT_VCF2" || {
    echo "Error: Indexing SNP file failed" >&2
    exit 1
}

# Step 6: Calculate allele frequencies
echo -e "######################\nCalculating allele frequencies"
vcftools --gzvcf "$OUTPUT_VCF" --freq --out "${ALLELE_FREQ_PATH}/AF" --max-alleles 2 || {
    echo "Error: Calculating allele frequencies failed for $OUTPUT_VCF" >&2
}
vcftools --gzvcf "$OUTPUT_VCF" --freq2 --out "${ALLELE_FREQ_PATH}/AF_2" --max-alleles 2 || {
    echo "Error: Calculating allele frequencies failed for $OUTPUT_VCF" >&2
}

echo "BCF and VCF files generated successfully."


In [ ]:
## Lancer le script

sbash snp_calling_pipeline.sh

### Correction de l'identité de chaque isolat dans les fichiers vcf

In [ ]:
## Se déplacer dans le répertoire SNP

cd /scratch/MOryzae/SNP/vcf_files

In [ ]:
## Charger le module bcftools

module load bcftools/1.18

In [ ]:
## Utilisez bcftools pour lister les noms des échantillons actuels :

bcftools query -l all_samples.vcf.gz > samples.txt

bcftools query -l all_samples_snp.vcf.gz> snp_samples.txt


Cela créera un fichier samples.txt contenant les noms des échantillons actuels (tels que ceux avec des chemins et suffixes).

In [ ]:
## Modifiez le fichier samples.txt ou créez un nouveau fichier, par exemple new_samples.txt, 
## en mappant les anciens noms aux nouveaux noms simplifiés.

awk -F'/' '{print $0 "\t" $NF}' samples.txt | sed 's/.mappedpaired.sorted.bam//g' > new_samples.txt

awk -F'/' '{print $0 "\t" $NF}' snp_samples.txt | sed 's/.mappedpaired.sorted.bam//g' > new_snp_samples.txt


In [ ]:
## Récuper rien que les noms des isolats dans un nouveau fichier texte

awk -F'/' '{print $0 "\t" $NF}' new_samples.txt | cut -f3 > new2_samples.txt

awk -F'/' '{print $0 "\t" $NF}' new_snp_samples.txt | cut -f3 > new2_snp_samples.txt


In [ ]:
### Utilisez bcftools reheader pour appliquer les modifications :

bcftools reheader -s new2_samples.txt -o all_samples_correct.vcf.gz all_samples.vcf.gz

bcftools reheader -s new2_snp_samples.txt -o all_samples_snp_correct.vcf.gz all_samples_snp.vcf.gz


Vérifier les modifications

In [ ]:
## Pour vérifier que les nouveaux noms ont été appliqués correctement :

bcftools query -l all_samples_correct.vcf.gz

bcftools query -l all_samples_snp_correct.vcf.gz


In [ ]:
## Reindexer les nouveaux fichiers vcf.gz

bcftools index all_samples_correct.vcf.gz

bcftools index all_samples_snp_correct.vcf.gz

### Récupérer tous les outputs générés dans le répertoire SNP sur le NAS

In [ ]:
## Déplacer tout le répertoire MAPPING vers le NAS

scp -r /scratch/MOryzae/SNP san:/projects/medium/CIBiG_MOryzae/